# Set up

In [4]:
# update path with data dir
import sys
sys.path.append('../data/')
sys.path.append('../modelling/')

In [5]:
import player_data as player
from neural_nets import goals
import pandas as pd
from datetime import datetime
import numpy as np
import patsy

/home/tom/footbot/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Getting data

In [6]:
file_name = 'csvs/element_gameweek_' + datetime.today().strftime('%Y-%m-%d') + '.csv'

In [7]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [8]:
current_event = 33
element_types = [1, 2, 3, 4]
threshold_minutes = 1

In [9]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
    & (element_gameweek_df['element_type'].isin(element_types))
    & (element_gameweek_df['minutes'] >= threshold_minutes)
]

# Modelling

## Goals

In [10]:
goals_formula = 'goals_scored ~ was_home + C(element) + C(opposition_team)'

In [11]:
goals_response, goals_explanatory =\
patsy.dmatrices(goals_formula, element_gameweek_df, return_type='dataframe')

In [12]:
goals_model = goals.train_model(goals_explanatory, goals_response)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
8967/8967 [==============================] - 2s 176us/sample - loss: 0.1116
Epoch 2/20
8967/8967 [==============================] - 1s 140us/sample - loss: 0.0993
Epoch 3/20
8967/8967 [==============================] - 1s 142us/sample - loss: 0.0993
Epoch 4/20
8967/8967 [==============================] - 1s 138us/sample - loss: 0.0992
Epoch 5/20
8967/8967 [==============================] - 1s 139us/sample - loss: 0.0992
Epoch 6/20
8967/8967 [==============================] - 1s 139us/sample - loss: 0.0992
Epoch 7/20
8967/8967 [==============================] - 1s 142us/sample - loss: 0.0991
Epoch 8/20
8967/8967 [==============================] - 1s 139us/sample - loss: 0.0990
Epoch 9/20
8967/8967 [==============================] - 1s 139us/sample - loss: 0.0990
Epoch 10/20
8967/8967 [==============================] - 1s 138us/sample - loss: 0.0989
Epoch 11/

# Predicting points

In [13]:
gameweek_fixtures_df = player.get_gameweek_fixtures_df(35, threshold_minutes, element_gameweek_df)

In [14]:
gameweek_fixtures_explanatory_df = player.get_gameweek_fixtures_explanatory_df(gameweek_fixtures_df)

## Goals

In [15]:
predicted_goals = goals.predict(gameweek_fixtures_explanatory_df, goals_model)

In [18]:
gameweek_fixtures_df['predicted_goals_scored'] = predicted_goals

In [20]:
gameweek_fixtures_df['predicted_goals_scored'].describe()

count    691.000000
mean       0.005985
std        0.045803
min       -0.002461
25%        0.000050
50%        0.000050
75%        0.000050
max        1.035761
Name: predicted_goals_scored, dtype: float64

In [21]:
gameweek_fixtures_df[gameweek_fixtures_df['predicted_goals_scored'] > 0.5]

,own_team,opposition_team,element,was_home,element_type,predicted_goals_scored
338,18,10,376,0,2,1.035761
